In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#!pip install fasttext

     |████████████████████████████████| 71kB 2.5MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3020358 sha256=d33afeed725a4b59d4ceab5a5f07569c606c9d4a0e2762ac14ae1550e52a1154
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
from collections import Counter
from skimage import io, transform
import torch.backends.cudnn as cudnn

from torchsummary import summary

import matplotlib.pyplot as plt # for plotting
import numpy as np
from time import time
import collections
import gensim

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
print("Let's use", torch.cuda.device_count(), "GPUs!")
parallel = True
platform = "colab"
restore = False

cpu
Let's use 0 GPUs!


In [1]:
phase = "Train"
VOCAB = {}
WORD2IDX = {}
IDX2WORD = {}

### Image Transforms

In [2]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, image):
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)
        img = transform.resize(image, (new_h, new_w))
        return img


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, image):
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return image


IMAGE_RESIZE = (256, 256)
# Sequentially compose the transforms
img_transform = transforms.Compose([Rescale(IMAGE_RESIZE), ToTensor()])


NameError: ignored

### Captions Preprocessing

In [ ]:
class CaptionsPreprocessing:
    """Preprocess the captions, generate vocabulary and convert words to tensor tokens
    Args:
        captions_file_path (string): captions tsv file path
    """
    def __init__(self, captions_file_path):
        self.captions_file_path = captions_file_path

        # Read raw captions
        self.raw_captions_dict = self.read_raw_captions()

        # Preprocess captions
        self.captions_dict = self.process_captions()

        # Create vocabulary
        self.start = "<start>"
        self.end = "<end>"
        self.oov = "<unk>"
        self.pad = "<pad>"
        self.vocab = self.generate_vocabulary()
        self.word2index = self.convert_word2index()        
        self.index2word = self.convert_index2word()
        self.max_len_caption = 50

    def read_raw_captions(self):
        """
        Returns:
            Dictionary with raw captions list keyed by image ids (integers)
        """
        captions_dict = {}
        with open(self.captions_file_path, 'r', encoding='utf-8') as f:
            for img_caption_line in f.readlines():
                img_captions = img_caption_line.strip().split('\t')
                captions_dict[int(img_captions[0])] = img_captions[1:]

        return captions_dict 

    def process_captions(self):
        """
        Use this function to generate dictionary and other preprocessing on captions
        """

        raw_captions_dict = self.raw_captions_dict 
        
        # Do the preprocessing here                
        captions_dict = raw_captions_dict

        return captions_dict

 

    def generate_vocabulary(self):
        """
        Use this function to generate dictionary and other preprocessing on captions
        """
        captions_dict = self.captions_dict

        # Generate the vocabulary
        
        all_captions = ""        
        for cap_lists in captions_dict.values():
            all_captions += " ".join(cap_lists)
        all_captions = all_captions.lower().replace(".", "").split(" ")
        
        vocab = {self.pad :1, self.oov :1, self.start :1, self.end :1}
        vocab_update = Counter(all_captions) 
        vocab_update = {k:v for k,v in vocab_update.items() if v >= freq_threshold}
        vocab.update(vocab_update)
        vocab_size = len(vocab)   
        VOCAB.update(vocab)
        print("VOCAB SIZE =", vocab_size)
        return vocab
    
    def convert_word2index(self):
        word2index = {}
        vocab = self.vocab
        idx = 0
        for k, v in vocab.items():
            word2index[k] = idx
            idx +=1
        
        return word2index
    
    def convert_index2word(self):
        index2word = {}
        vocab = self.vocab
        idx = 0
        
        for k, v in vocab.items():
            index2word[idx] = k
            idx +=1
        
        return index2word

 

    def captions_transform(self, img_caption_list, max_len):
        """
        Use this function to generate tensor tokens for the text captions
        Args:
            img_caption_list: List of captions for a particular image
        """
        word2index = self.word2index
        vocab = self.vocab
        #index2word = self.index2word        
        #embed = self.embed
        start = self.start
        end = self.end
        oov = self.oov
        #print("MX LEN", max_len)
        processed_list = list(map(lambda x: start + " "+ x + " " + end, img_caption_list))
        processed_list = list(map(lambda x: x.lower().replace(".", "").split(" "), processed_list))
        processed_list = list(map(lambda x: list(map(lambda y: word2index[y] if y in vocab else word2index[oov],x)),
                                  processed_list))
        processed_list = list(map(lambda x: x + ( [0] * int(max_len - len(x) + 2) ),processed_list))
        
        # Generate tensors
        #print(np.array(processed_list).shape)
        processed_list = torch.LongTensor(processed_list)
        #processed_captions = embed(processed_list)   
        #print(processed_captions)    
        
        #return torch.zeros(len(img_caption_list), 10)
        return processed_list


if platform == "colab":
    CAPTIONS_FILE_PATH = '/content/drive/My Drive/A4/train_captions.tsv'
else:
    CAPTIONS_FILE_PATH = "../data/train_captions.tsv"
    
embedding_dim = 256
freq_threshold = 4
captions_preprocessing_obj = CaptionsPreprocessing(CAPTIONS_FILE_PATH)

VOCAB SIZE = 10700


### Dataset Class

In [ ]:
class ImageCaptionsDataset(Dataset):

    def __init__(self, img_dir, captions_dict, img_transform=None, captions_transform=None):
        """
        Args:
            img_dir (string): Directory with all the images.
            captions_dict: Dictionary with captions list keyed by image ids (integers)
            img_transform (callable, optional): Optional transform to be applied
                on the image sample.

            captions_transform: (callable, optional): Optional transform to be applied
                on the caption sample (list).
        """
        self.img_dir = img_dir
        self.captions_dict = captions_dict
        self.img_transform = img_transform
        self.captions_transform = captions_transform

        self.image_ids = list(captions_dict.keys())
        self.max_len = max([max(list(map(lambda x: len(x.lower().replace(".", "").split(" ")), v))) for k,v in captions_dict.items()])
        

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, 'image_{}.jpg'.format(self.image_ids[idx]))
        image = io.imread(img_name)
        captions = self.captions_dict[self.image_ids[idx]]
        if self.img_transform:
            image = self.img_transform(image)

        if self.captions_transform:            
            captions = self.captions_transform(captions, self.max_len)

        sample = {'image': image, 'captions': captions}

        return sample

### Prepare Weights from GloVe

In [ ]:
!wget "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.vec.gz" -P "/content/drive/My Drive/wordvec/"

--2020-09-05 18:07:22--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1272825284 (1.2G) [binary/octet-stream]
Saving to: ‘/content/drive/My Drive/wordvec/cc.it.300.vec.gz’

cc.it.300.vec.gz    100%[===================>]   1.18G  14.7MB/s    in 1m 45s  

2020-09-05 18:09:08 (11.5 MB/s) - ‘/content/drive/My Drive/wordvec/cc.it.300.vec.gz’ saved [1272825284/1272825284]



In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/A4/wordvec/52.zip', 'r') as zip_ref:
    zip_ref.extractall("/content/drive/My Drive/A4/wordvec/52/")

In [ ]:
model159 = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/A4/wordvec/159/model.txt', binary=False, unicode_errors='replace')

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/A4/wordvec/52/model.txt', binary=False, unicode_errors='replace')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
model.vocab

In [ ]:
fastmodel = gensim.models.wrappers.FastText.load_fasttext_format('/content/drive/My Drive/A4/wordvec/cc.it.300.vec.gz', binary=True, unicode_errors='replace')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
fastWikimodel = gensim.models.('/content/drive/My Drive/wordvec/wiki.it.vec', binary=True, unicode_errors='replace')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
FastTextItalian = gensim.models.wrapp .load_word2vec_format('/content/drive/My Drive/wordvec/cc.it.300.vec.gz', binary=False, unicode_errors='replace')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
for k, v in FastTextItalian.vocab.items():
  wiki_dict[k.lower()] = FastTextItalian.word_vec(k)


In [ ]:
len(FastTextItalian.vocab.keys())

2000000

In [ ]:
count = 0
for k,v in VOCAB.items():
  if k.replace(",","").replace("'","") not in wiki_dict:
    count+=1
    print(k)

count

<pad>
<unk>
<start>
<end>
insiemediversi
finestradue
chitarraun
edificiodue
scalaun
un'uniforme
all'esternouna
nell'erba
nell'erbaun
luiun
strumentidue
strumentiuna
ponteun
crowda
metropolitanaun
pratoun
sull'angolo
stradauna
all'apertoun
sull'estremità
sull'autobus
fuoriuna
sull'erba
accantoun
bottigliaun
stradaun
sull'albero
biciclettaun
nevel'uomo
luceuna
scoglieraun
giocattolouna
aereoun
tuboun
sorridendoun
giallaun
tavoloun
campodue
neveun
un'anatra
nell'acquaun
ghiacciodue
spiaggiaun
matrimonioun
boccaun
concertouna
skateparkun
gialloun
sodoun
stradadue
dell'orgoglio
d'arcobaleno
bandiereun
marroneun
goalista
all'allenatore
statuaun
carrozzinaun
prodottiun
ciboun
pallaun
elettricaun
caucasiana
cenauna
erbosoun
fuoritre
verdeun
stagnoun
anzianoun
musicaleun
all'apertouna
biciun
violaun
testaun
rocciosoun
d'artedue
flip-flops
giocattoloun
sull'erbauna
un'imbarcazione
nell'oceano
surfdue
ristorantedue
costruzioneun
sentieroun
legnouna
macchinauna
fucileun
un'attrezzatura
macchinariu

1395

In [ ]:
for k in wiki_dict.keys():
  if "<o" in k:
    print(k)

<ol>
<option
<options>
><object
<oroblu.it>
<oroblu.org>
<o>
<outfile>
<object>
<<obiettivi
<onlyinclude>
<opzioni
<<ora
<option>
<o:p></
option><option
<<oh
<ogni
<o
<obbligatorio>
<ora
<<o
<object
p<o
<organi
<oaf_attribute
?<odiusseus>
<olivier.crete@collabora.com>date
<oroblu.biz>
<operation
<optgroup>
<ourhdr.h>se
<omissis>
carige><obiettivo
<oh


In [ ]:
wiki_dict = {}

### Encoder

In [ ]:
vocab_w2v = model.vocab

In [ ]:
for k,v in vocab_w2v.items():
  if "" in k:
    print(k, v)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, embed_dim):
        super(Encoder, self).__init__()
        resnet152 = models.resnet152(pretrained=True, progress=True)
        for param in resnet152.parameters():
            param.requires_grad = False
        print("ResNet152 Loaded Successfully..!")
        self.ResNet152 = resnet152
        #self.adaptive_pool = nn.AdaptiveAvgPool2d(output_size = (1,1))
        self.fc = nn.Linear(in_features=1000, out_features=embed_dim, bias = True)
    
    def forward(self, x):
        x = self.ResNet152(x)
        x = self.fc(x)
        return x

### Decoder

In [ ]:
class AttentionBlock(nn.Module):
    def __init__(self, embed_dim, units, vocab_size):
        super(AttentionBlock, self).__init__()
        self.W1 = nn.Linear(in_features = embed_dim, out_features = units)
        self.W2 = nn.Linear(in_features=units, out_features=units)
        self.V = nn.Linear(in_features=units, out_features=1)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, img_features, hidden):
        
        hidden = hidden.unsqueeze(dim=1)
        hidden = hidden.double()
        #print("feature and hidden shape",img_features.shape, hidden.shape)
        combined_score = self.tanh(self.W1(img_features) + self.W2(hidden))
        
        attention_weights = self.softmax(self.V(combined_score))
        context_vector = attention_weights * img_features
        context_vector = torch.sum(context_vector, dim=1)
        
        return context_vector, attention_weights    



class Decoder(nn.Module):
    def __init__(self, embed_dim, lstm_hidden_size,lstm_layers=1):
        super(Decoder, self).__init__()
        self.lstm_hidden_size = lstm_hidden_size
        self.vocab_size = len(VOCAB)
        print("VOCAB SIZE = ", self.vocab_size)
        
        self.lstm = nn.LSTM(input_size = embed_dim, hidden_size = lstm_hidden_size,
                            num_layers = lstm_layers, batch_first = True)
        
        self.linear = nn.Linear(lstm_hidden_size, self.vocab_size)        
        self.embed = nn.Embedding(self.vocab_size, embed_dim)
        
        self.attention = AttentionBlock(embed_dim, lstm_hidden_size, self.vocab_size)

        
    def forward(self, img_features, image_captions):
        image_features = torch.Tensor.repeat_interleave(img_features, repeats=5 , dim=0)
        image_features = image_features.unsqueeze(1)
        
        hidden = torch.zeros((image_features.shape[0], self.lstm_hidden_size))
        
        context, attention = self.attention(image_features, hidden)
        
        embedded_captions = self.embed(image_captions)
        #print("EMBED SHAPE", embedded_captions.shape)
        #print("SHAPES BEFORE CONCAT",context.unsqueeze(dim=1).shape, embedded_captions[:,1:].shape)
        input_lstm = torch.cat((context.unsqueeze(dim=1), embedded_captions[:,1:]), dim = 1)
        
        lstm_outputs, _ = self.lstm(input_lstm)        
        lstm_outputs = self.linear(lstm_outputs)
        #print("lstm_outputs.shape", lstm_outputs.shape)
        
        
        return lstm_outputs



In [ ]:
def create_checkpoint(path,model, optim_obj, loss_obj,iteration, epoch):
    checkpoint = {'epoch': epoch,
                  'iteration': iteration,
                  'model_state_dict': model.module.state_dict(),
                  'optimizer_state_dict': optim_obj.state_dict(),
                  'loss': loss_obj}

    if platform == "colab":
        directory = '/content/drive/My Drive/A4/comp_checkpoint/'
    else:
        directory = '../comp_checkpoint/'

    torch.save(checkpoint, directory + path)
    
def restore_checkpoint(path):
    new_state_dict = collections.OrderedDict()
    if platform == "colab":
        directory = '/content/drive/My Drive/A4/comp_checkpoint/'
        checkpoint = torch.load(directory + path)
    else:
        directory = '../comp_checkpoint/'
        checkpoint = torch.load(directory + path, map_location=torch.device('cpu'))    
    
    epoch = checkpoint['epoch']
    new_state_dict = checkpoint['model_state_dict']
    iteration = checkpoint['iteration']
    optimizer_state_dict = checkpoint['optimizer_state_dict']
    loss_obj = checkpoint['loss']
    print("Iterations = {}, Epoch = {}, loss = {}".format(iteration, epoch, loss_obj.item()))
    return new_state_dict

In [ ]:
class ImageCaptionsNet(nn.Module):
    def __init__(self):
        super(ImageCaptionsNet, self).__init__()

        # Define your architecture here
        
        ##CNN ENCODER RESNET-50
        
        self.Encoder = Encoder(embed_dim = embedding_dim)
        self.Decoder = Decoder(embedding_dim, units, 1)        

    def forward(self, x):
        x = image_batch, captions_batch

        # Forward Propogation
        x = self.Encoder(image_batch)
        #print("Encoder OP shape", x.shape)
        x = self.Decoder(x, captions_batch)
        return x
    

units = 512
if restore == False:
    net = ImageCaptionsNet()
    net = net.double()
    # If GPU training is required
    if parallel == True and device != "cpu":
        print("Parallel Processing enabled")
        net = nn.DataParallel(net)

    if device == "cpu":
        print("Device to CPU")
    else:
        print("Device to CUDA")
        net = net.to(torch.device("cuda:0"))



### Training Loop

In [ ]:
if platform == "colab":
    IMAGE_DIR = '/content/drive/My Drive/train_images/'
else:
    IMAGE_DIR = 'D:/Padhai/IIT Delhi MS(R)/2019-20 Sem II/COL774 Machine Learning/Assignment/Assignment4/train_images/'

if restore == True:
    net = ImageCaptionsNet()
    net = net.double()
    #net = net.to(torch.device("cuda:0"))
    #net = nn.DataParallel(net)
    state_dict = restore_checkpoint("caption_chkpt_multi.pth")
    '''new_state_dict = collections.OrderedDict()
    for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v'''

    # load params
    net.load_state_dict(state_dict)
    print("State Dictionary Loaded Successfully.")
    net = nn.DataParallel(net)
    net = net.to(torch.device("cuda:0"))

# Creating the Dataset
train_dataset = ImageCaptionsDataset(
    IMAGE_DIR, captions_preprocessing_obj.captions_dict, img_transform=img_transform,
    captions_transform=captions_preprocessing_obj.captions_transform
)

# Define your hyperparameters
NUMBER_OF_EPOCHS = 3
LEARNING_RATE = 1e-1
BATCH_SIZE = 2
NUM_WORKERS = 0 # Parallel threads for dataloading
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE)
loss_list = []
# Creating the DataLoader for batching purposes
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
import os
if device != "cpu":
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
    cudnn.benchmark = True
t0 = time()
for epoch in range(NUMBER_OF_EPOCHS):
    print("$$$$$----EPOCH {}----$$$$$$".format(epoch+3))
    iteration = 0
    for batch_idx, sample in enumerate(train_loader):
        iteration +=1
        net.zero_grad()

        image_batch, captions_batch = sample['image'], sample['captions']
        
        #print("image_shape", image_batch.shape)
        #print("caption_shape", captions_batch.shape)
        
        N, I, L = captions_batch.shape
        captions_batch = torch.reshape(captions_batch, (N*I,L))

        # If GPU training required
        if device != "cpu":
            image_batch, captions_batch = image_batch.to(torch.device("cuda:0")), captions_batch.to(torch.device("cuda:0"))
        #print("Running Caption Gen")
        output_captions = net((image_batch, captions_batch))
        #print("output Achieved")

        #print("size before loss", output_captions.shape, captions_batch.shape)
        #print("size prepared for loss", output_captions.reshape(-1, output_captions.shape[2]).shape, captions_batch.reshape(-1).shape)
        loss = loss_function(output_captions.reshape(-1, output_captions.shape[2]), captions_batch.reshape(-1))
        loss_list.append(loss.item())
        
        loss.backward()
        optimizer.step()
        #print("creating checkpoint")
        if iteration%5 == 0:
            create_checkpoint("caption_chkpt_multi.pth", net, optimizer, loss, iteration, epoch+3)
        print("ITERATION:[{}/{}] | LOSS: {} | EPOCH = [{}/{}] | TIME ELAPSED ={}Mins".format(iteration, round(29000/BATCH_SIZE),
              round(loss.item(), 6), epoch+3, NUMBER_OF_EPOCHS, round((time()-t0)/60,2)))
    print("\n$$Loss = {},EPOCH: [{}/{}]\n\n".format(round(loss.item(), 6), epoch+3, NUMBER_OF_EPOCHS))
    create_checkpoint("epoch{}_chkpt_multi.pth".format(epoch+3), net, optimizer, loss, iteration, epoch+3)

create_checkpoint("Final_Model_multi.pth", net, optimizer, loss, iteration, epoch+3)


### Test Loop

In [ ]:
if platform == "colab":
    IMAGE_DIR = '/content/drive/My Drive/public_test_images/'
else:
    IMAGE_DIR = 'D:/Padhai/IIT Delhi MS(R)/2019-20 Sem II/COL774 Machine Learning/Assignment/Assignment4/train_images/'
restore = True
if restore == True:
    net = ImageCaptionsNet()
    net = net.double()
    #net = net.to(torch.device("cuda:0"))
    #net = nn.DataParallel(net)
    state_dict = restore_checkpoint("caption_chkpt_multi.pth")
    '''new_state_dict = collections.OrderedDict()
    for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v'''
    
    net.load_state_dict(state_dict)
    print("State Dictionary Loaded Successfully.")

    # load params
    if device != 'cpu':
        net = nn.DataParallel(net)
        net = net.to(torch.device("cuda:0"))


# Creating the Dataset
test_dataset = ImageCaptionsDataset(
    IMAGE_DIR, captions_preprocessing_obj.captions_dict, img_transform=img_transform,
    captions_transform=captions_preprocessing_obj.captions_transform
)

# Define your hyperparameters
NUMBER_OF_EPOCHS = 1
LEARNING_RATE = 1e-1
BATCH_SIZE = 2
NUM_WORKERS = 0 # Parallel threads for dataloading
# Creating the DataLoader for batching purposes
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
import os
if device != "cpu":
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
t0 = time()
for batch_idx, sample in enumerate(test_loader):
        image_batch, captions_batch = sample['image'], sample['captions']
        img_features = net.Encoder(image_batch)
        #print(x.shape)
        

In [ ]:
z = torch.randn((2,3,4))

In [ ]:
type(z.shape[0])